---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram

def graph_identification():
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_hist = degree_distribution(G)
        if len(degree_hist)>10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    return methods
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [67]:
def salary_predictions():
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    def is_management(node):
        management = node[1]['ManagementSalary']
        if management == 1.0:
            return 1
        elif management == 0:
            return 0
        else:
            return None
        
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(nx.degree(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['is_management'] = [is_management(node) for node in G.nodes(data=True)]
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized = True))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['pagerank'] = pd.Series(nx.pagerank(G))
    
    df_train = df[(df['is_management'] == 0) | (df['is_management'] == 1)]
    df_test = df[(df['is_management'] != 0) & (df['is_management'] != 1)]
    
    
    features = ['Department', 'clustering', 'degree', 'degree_centrality', 'betweenness', 'closeness', 'pagerank']
    
    X_train = df_train[features]
    y_train = df_train['is_management']
    X_test = df_test[features]
    
    clf = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
    probability = clf.predict_proba(X_test)[:,1]
    
    return pd.Series(probability, X_test.index)
salary_predictions()

1       0.0
2       1.0
5       1.0
8       0.4
14      0.0
18      0.1
27      0.1
30      0.4
31      0.7
34      0.0
37      0.1
40      0.0
45      0.0
54      0.4
55      0.5
60      0.4
62      1.0
65      1.0
77      0.0
79      0.0
97      0.0
101     0.0
103     0.5
108     0.0
113     0.1
122     0.0
141     0.0
142     1.0
144     0.0
145     0.6
       ... 
913     0.0
914     0.0
915     0.0
918     0.0
923     0.0
926     0.1
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.0
950     0.0
951     0.0
953     0.0
959     0.0
962     0.0
963     0.0
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [23]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [26]:
def new_connections_predictions():
    from sklearn.ensemble import RandomForestClassifier
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential attachment'] = [x[2] for x in preferential_attachment]
    df['cn_soundarajan_hopcroft'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G))]
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df = future_connections.join(df, how = 'outer')
    
    df_train = df[(df['Future Connection'] == 0) | (df['Future Connection'] == 1)]
    df_test = df[(df['Future Connection'] != 0) & (df['Future Connection'] != 1)]
    
    
    features = ['preferential attachment', 'cn_soundarajan_hopcroft', 'resource_allocation_index', 'jaccard_coefficient']
    
    X_train = df_train[features]
    y_train = df_train['Future Connection']
    X_test = df_test[features]
    
    clf = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
    probability = clf.predict_proba(X_test)[:,1]
    
    return pd.Series(probability, X_test.index)

new_connections_predictions()

(0, 9)          0.000000
(0, 19)         0.000000
(0, 20)         0.300000
(0, 35)         0.000000
(0, 38)         0.000000
(0, 42)         0.900000
(0, 43)         0.000000
(0, 50)         0.000000
(0, 53)         0.900000
(0, 54)         0.000000
(0, 62)         0.100000
(0, 63)         0.000000
(0, 69)         0.000000
(0, 72)         0.010021
(0, 83)         0.100000
(0, 90)         0.200000
(0, 91)         0.000000
(0, 95)         0.100000
(0, 100)        0.000000
(0, 106)        1.000000
(0, 108)        0.000000
(0, 109)        0.013252
(0, 110)        0.021011
(0, 118)        0.000000
(0, 122)        0.200000
(0, 131)        0.100000
(0, 133)        0.900000
(0, 140)        0.200000
(0, 149)        0.300000
(0, 151)        0.000000
                  ...   
(988, 989)      0.008810
(988, 996)      0.015677
(988, 997)      0.100000
(988, 1000)     0.015973
(988, 1002)     0.015677
(989, 994)      0.013263
(989, 996)      0.013263
(989, 1003)     0.013263
(989, 1004)     0.013263
